# Exploratory Data Analysis with a small subset of data
- Use a subset of major directors -> around 20
- subset those directors to their films
- subset ratings and etc
- need to explor incorporting OMDB to get the ratings and view various ranking systems on good films


#### Load Packages

In [1]:
import pandas as pd
import polars as pl
import gzip
import pyarrow
import sys
from pathlib import Path
from tqdm import tqdm
import os


import matplotlib.pyplot as plt
import seaborn as sns
import sys
from pathlib import Path

# Add the `src` folder to Python's search path
src_path = "/Users/bradkittrell/Projects/imdb/IMDB_Fun/src"
sys.path.append(src_path)

In [2]:
from data import generate_dataset

import json

# Load the JSON file
with open("/Users/bradkittrell/Projects/imdb/IMDB_Fun/src/data/api_keys.json", "r") as json_file:
    config = json.load(json_file)

config['OMDB_API_KEY']

'95497563'

## Load Data and merge

In [3]:
# going to use imdbs list of top 25 directors of all time

names = generate_dataset.load_names(None)
film_crew = generate_dataset.load_crew(None)
films = generate_dataset.load_title_basics(None)

INFO:root:No path provided. Using default path: /Users/bradkittrell/Projects/imdb/IMDB_Fun/Data/processed/name.parquet
INFO:root:Successfully loaded data from /Users/bradkittrell/Projects/imdb/IMDB_Fun/Data/processed/name.parquet
INFO:root:Filtered names data to include only rows where birthYear > 1899
INFO:root:Dropped __index_level_0__ column
INFO:root:No path provided. Using default path: /Users/bradkittrell/Projects/imdb/IMDB_Fun/Data/processed/title_crew.parquet
INFO:root:Successfully loaded data from /Users/bradkittrell/Projects/imdb/IMDB_Fun/Data/processed/title_crew.parquet
INFO:root:No path provided. Using default path: /Users/bradkittrell/Projects/imdb/IMDB_Fun/Data/processed/title_basics_movies.parquet
INFO:root:Successfully loaded data from /Users/bradkittrell/Projects/imdb/IMDB_Fun/Data/processed/title_basics_movies.parquet
INFO:root:Dropped __index_level_0__ column


In [4]:

top_n_directors = ["Steven Spielberg", "Martin Scorsese", "Alfred Hitchcock", "Roman Polanksi",
                   "John Ford", "Billy Wilder", "James Cameron", "Tim Burton", "Ridley Scott", "Joel Coen", "David Fincher",
                   "Christopher Nolan", "Clint Eastwood", "David Lean", "Woody Allen", "Stanley Kubrick", "Danny Boyle", "George Lucas", "Peter Jackson", "Quentin Tarantino",
                   "Fancis Ford Coppola"]
# lots of duplicate names or same names rather
top_n_directors_names = names.filter((pl.col("primaryName").is_in(
    top_n_directors)) & (pl.col("nconst") != "nm13407303") & (pl.col("nconst") != "nm0131607") & (pl.col("nconst") != "nm0131609") & (pl.col("nconst") != "nm7812401"))[["nconst", "primaryName"]]

top_n_directors_names


film_crew = film_crew.filter(
    pl.col("directors").is_in(top_n_directors_names["nconst"]))

top_n_directors_names_films = film_crew.join(top_n_directors_names, left_on="directors",
                                             right_on="nconst", how="inner")[["tconst", "primaryName"]]

top_n_directors_names_films = top_n_directors_names_films.join(
    films, on="tconst")

In [6]:
top5 = top_n_directors_names_films['tconst'].head().to_list()[0:5]


# generate_dataset.fetch_movie_data(imdb_film_id=top5[0], api_key="95497563")
generate_dataset.process_omdb_ids(top5)

INFO:root:No path provided. Using default path: /Users/bradkittrell/Projects/imdb/IMDB_Fun/Data/external/omdb.csv


,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,RatingSource,RatingValue
0,Once Upon a Time... in Hollywood,2019,R,26 Jul 2019,161 min,"Comedy, Drama",Quentin Tarantino,Quentin Tarantino,"Leonardo DiCaprio, Brad Pitt, Margot Robbie",Quentin Tarantino's Once Upon a Time... in Hol...,...,"882,682",tt7131622,movie,NaN,"$142,502,728",NaN,NaN,True,Internet Movie Database,7.6/10
1,Once Upon a Time... in Hollywood,2019,R,26 Jul 2019,161 min,"Comedy, Drama",Quentin Tarantino,Quentin Tarantino,"Leonardo DiCaprio, Brad Pitt, Margot Robbie",Quentin Tarantino's Once Upon a Time... in Hol...,...,"882,682",tt7131622,movie,NaN,"$142,502,728",NaN,NaN,True,Rotten Tomatoes,86%
2,Once Upon a Time... in Hollywood,2019,R,26 Jul 2019,161 min,"Comedy, Drama",Quentin Tarantino,Quentin Tarantino,"Leonardo DiCaprio, Brad Pitt, Margot Robbie",Quentin Tarantino's Once Upon a Time... in Hol...,...,"882,682",tt7131622,movie,NaN,"$142,502,728",NaN,NaN,True,Metacritic,84/100
3,Ace in the Hole,1951,Approved,04 Jul 1951,111 min,"Drama, Film-Noir",Billy Wilder,"Billy Wilder, Lesser Samuels, Walter Newman","Kirk Douglas, Jan Sterling, Robert Arthur","Charles Tatum, a down-on-his-luck reporter, ta...",...,"40,295",tt0043338,movie,NaN,NaN,NaN,NaN,True,Internet Movie Database,8.1/10
4,Ace in the Hole,1951,Approved,04 Jul 1951,111 min,"Drama, Film-Noir",Billy Wilder,"Billy Wilder, Lesser Samuels, Walter Newman","Kirk Douglas, Jan Sterling, Robert Arthur","Charles Tatum, a down-on-his-luck reporter, ta...",...,"40,295",tt0043338,movie,NaN,NaN,NaN,NaN,True,Rotten Tomatoes,90%
5,Ace in the Hole,1951,Approved,04 Jul 1951,111 min,"Drama, Film-Noir",Billy Wilder,"Billy Wilder, Lesser Samuels, Walter Newman","Kirk Douglas, Jan Sterling, Robert Arthur","Charles Tatum, a down-on-his-luck reporter, ta...",...,"40,295",tt0043338,movie,NaN,NaN,NaN,NaN,True,Metacritic,72/100
6,The Sound Barrier,1952,Approved,21 Dec 1952,109 min,"Adventure, Drama, Romance",David Lean,Terence Rattigan,"Ralph Richardson, Ann Todd, Nigel Patrick","Tony, a successful World War II fighter pilot,...",...,"2,412",tt0044446,movie,NaN,NaN,NaN,NaN,True,Internet Movie Database,6.7/10
7,The Sound Barrier,1952,Approved,21 Dec 1952,109 min,"Adventure, Drama, Romance",David Lean,Terence Rattigan,"Ralph Richardson, Ann Todd, Nigel Patrick","Tony, a successful World War II fighter pilot,...",...,"2,412",tt0044446,movie,NaN,NaN,NaN,NaN,True,Rotten Tomatoes,80%
8,Fear and Desire,1952,Not Rated,01 Apr 1953,62 min,"Drama, Thriller, War",Stanley Kubrick,Howard Sackler,"Frank Silvera, Kenneth Harp, Paul Mazursky",A ficticious war in an unidentified country pr...,...,"14,167",tt0045758,movie,NaN,NaN,NaN,NaN,True,Internet Movie Database,5.3/10
9,Fear and Desire,1952,Not Rated,01 Apr 1953,62 min,"Drama, Thriller, War",Stanley Kubrick,Howard Sackler,"Frank Silvera, Kenneth Harp, Paul Mazursky",A ficticious war in an unidentified country pr...,...,"14,167",tt0045758,movie,NaN,NaN,NaN,NaN,True,Rotten Tomatoes,70%


In [5]:
import requests

#  http://www.omdbapi.com/?i=tt3896198&apikey=95497563


def fetch_movie_data(title, api_key):

    url = f"http://www.omdbapi.com/?i={title}&plot=full&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None


movie_data = fetch_movie_data('tt7131622', '95497563')
# print(movie_data)

In [6]:
from urllib import response


test_df = pd.DataFrame(movie_data)


def response_to_df(response):

    _ = pd.DataFrame(response)
    _['RatingSource'] = _["Ratings"].apply(lambda x: x.get('Source'))
    _['RatingValue'] = _["Ratings"].apply(lambda x: x.get('Value'))

    return _


response_to_df(movie_data).to_csv(
    "/Users/bradkittrell/Projects/imdb/IMDB_Fun/Data/external/omdb.csv", index=False)

In [11]:
response_to_df = pd.read_csv(
    "/Users/bradkittrell/Projects/imdb/IMDB_Fun/Data/external/omdb.csv")
response_to_df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,RatingSource,RatingValue
0,Once Upon a Time... in Hollywood,2019,R,26 Jul 2019,161 min,"Comedy, Drama",Quentin Tarantino,Quentin Tarantino,"Leonardo DiCaprio, Brad Pitt, Margot Robbie",Quentin Tarantino's Once Upon a Time... in Hol...,...,"882,682",tt7131622,movie,NaN,"$142,502,728",NaN,NaN,True,Internet Movie Database,7.6/10
1,Once Upon a Time... in Hollywood,2019,R,26 Jul 2019,161 min,"Comedy, Drama",Quentin Tarantino,Quentin Tarantino,"Leonardo DiCaprio, Brad Pitt, Margot Robbie",Quentin Tarantino's Once Upon a Time... in Hol...,...,"882,682",tt7131622,movie,NaN,"$142,502,728",NaN,NaN,True,Rotten Tomatoes,86%
2,Once Upon a Time... in Hollywood,2019,R,26 Jul 2019,161 min,"Comedy, Drama",Quentin Tarantino,Quentin Tarantino,"Leonardo DiCaprio, Brad Pitt, Margot Robbie",Quentin Tarantino's Once Upon a Time... in Hol...,...,"882,682",tt7131622,movie,NaN,"$142,502,728",NaN,NaN,True,Metacritic,84/100
